# Interactions with EdgeX Foundry Microservices

In this notebook, we will explore the functionality of EdgeX Foundy. EdgeX Foundry is a microservice oriented architecture that enables programatic integration of IoT devices on an edge network. EdgeX is specifically build with industrial and business use-cases in mind.

## API Reference
As we go through this lab, you will find it useful to refer to the API reference page.
https://docs.edgexfoundry.org/Ch-APIReference.html

## Loading the Requests Library

In [9]:
# Import the Python HTTP Requests Library
import requests

After loading the library, we can make our first HTTP request to demonstrate how to use the Python3 Requests library

In [22]:
r = requests.get('https://api.github.com/orgs/SSG-DRD-IOT/events')

In [23]:
# View the Python Response Object
r

<Response [200]>

In [24]:
# Get the HTTP status code that was returned with the request
r.status_code

200

In [25]:
# Test whether the status code was 200 OK, the HTTP success code.
r.status_code == requests.codes.ok

True

In [26]:
print(r.text)

[{"id":"8825900324","type":"PushEvent","actor":{"id":12846,"login":"agnathan","display_login":"agnathan","gravatar_id":"","url":"https://api.github.com/users/agnathan","avatar_url":"https://avatars.githubusercontent.com/u/12846?"},"repo":{"id":163895759,"name":"SSG-DRD-IOT/lab-workload-consolidation","url":"https://api.github.com/repos/SSG-DRD-IOT/lab-workload-consolidation"},"payload":{"push_id":3177014332,"size":1,"distinct_size":1,"ref":"refs/heads/master","head":"7ad41b6d2c1de2252d13c56ee4297e9bde61a714","before":"c6cdda8ca74c21776c49e68e2dd6810f4038bd01","commits":[{"sha":"7ad41b6d2c1de2252d13c56ee4297e9bde61a714","author":{"email":"agnathan@gmail.com","name":"Daniel Holmlund"},"message":"Initial commit of the Introduction to Workload Consolidation\n\nThis is the initial introduction to the Workload Consolidation labs. We should link all labs related to virtualization and real-time systems here.","distinct":true,"url":"https://api.github.com/repos/SSG-DRD-IOT/lab-workload-consolid

In [38]:
import json
def pjson(text):
    parsed = json.loads(text)
    print(json.dumps(parsed, indent=4, sort_keys=True))
    
pjson(r.text)

[
    {
        "address": "edgex-device-virtual",
        "baseURL": "HTTP://edgex-device-virtual:49990",
        "created": 1546554449151,
        "id": "5c2e8c519f8fc20001304878",
        "method": "POST",
        "modified": 0,
        "name": "edgex-device-virtual",
        "origin": 1546554448888,
        "password": null,
        "path": "/api/v1/callback",
        "port": 49990,
        "protocol": "HTTP",
        "publisher": null,
        "topic": null,
        "url": "HTTP://edgex-device-virtual:49990/api/v1/callback",
        "user": null
    },
    {
        "address": "edgex-device-virtual",
        "baseURL": "OTHER://edgex-device-virtual:49990",
        "created": 1546554452864,
        "id": "5c2e8c549f8fc20001304886",
        "method": "POST",
        "modified": 0,
        "name": "JC.RR5.NAE9.ConfRoom.Padre.Island01-virtual-addressable",
        "origin": 1546554452851,
        "password": null,
        "path": null,
        "port": 49990,
        "protocol": "OTHER

## Define your Data
### Addressables

The Device Service will often establish at least two Addressable objects with the Core Metadata micro service. An Addressable is a flexible EdgeX object that specifies a physical address of something - in this case the physical address of the Device Service and the Device (the camera). While an Addressable could be created for a named MQTT pipe or other protocol endpoint, for this example, we will assume that both the Device Service and Device are able to be reached via HTTP REST calls.

So in this case, the Device Service would make two calls to Core Metadata, to create the Addressable for the Device Service:

In [34]:
url = "http://localhost:48081/api/v1/addressable"

payload = "{\n\t\"name\":\"Camera Control\",\n\t\"protocol\":\"HTTP\",\n\t\"address\":\"172.17.0.1\",\n\t\"port\":49977,\n\t\"path\":\"/cameracontrol\",\n\t\"publisher\":\"none\",\n\t\"user\":\"none\",\n\t\"password\":\"none\",\n\t\"topic\":\"none\"\n}"
headers = {
    'cache-control': "no-cache",
    'Postman-Token': "054ef071-9e4b-4f87-bbaf-8486fcca5db0"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

5c2ff1029f8fc200013048ad


In [36]:
import requests

url = "http://localhost:48081/api/v1/addressable"

payload = "{\n\t\"name\":\"camera1 address\",\n\t\"protocol\":\"HTTP\",\n\t\"address\":\"172.17.0.1\",\n\t\"port\":49999,\n\t\"path\":\"/camera1\",\n\t\"publisher\":\"none\",\n\t\"user\":\"none\",\n\t\"password\":\"none\",\n\t\"topic\":\"none\"\n}"
headers = {
    'cache-control': "no-cache",
    'Postman-Token': "ecd7ce3a-1711-4bfa-a809-33a9e36d1d80"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

5c2ff1909f8fc200013048ae


Define the value descriptors

In [41]:
# First, list all the value descriptors

import requests

url = "http://localhost:48080/api/v1/valuedescriptor"

payload = ""
headers = {
    'cache-control': "no-cache",
    'Postman-Token': "55df389a-bac4-4976-8c9d-4c8bd8485aef"
    }

response = requests.request("GET", url, data=payload, headers=headers)


pjson(response.text)

[
    {
        "created": 1546554453291,
        "defaultValue": "true",
        "description": "whether generating random value in each collection cycle",
        "formatting": "%s",
        "id": "5c2e8c5553201a67a58f2cb2",
        "labels": null,
        "max": null,
        "min": null,
        "modified": 0,
        "name": "enableRandomization",
        "origin": 0,
        "type": "B",
        "uomLabel": "Random"
    },
    {
        "created": 1546554453376,
        "defaultValue": "15",
        "description": "the frequency of collection",
        "formatting": "%s",
        "id": "5c2e8c5553201a67a58f2cbe",
        "labels": null,
        "max": "600",
        "min": "0",
        "modified": 0,
        "name": "collectionFrequency",
        "origin": 0,
        "type": "I",
        "uomLabel": "Seconds"
    },
    {
        "created": 1546554453326,
        "defaultValue": "0.00",
        "description": "The current temperature.",
        "formatting": "%s",
        "id": "

In [43]:
# Create the ValueDescriptor for the "humancounter"
import requests

url = "http://localhost:48080/api/v1/valuedescriptor"

payload = "{\n\t\"name\":\"humancount\",\n\t\"description\":\"people count\", \n\t\"min\":\"0\",\n\t\"max\":\"100\",\n\t\"type\":\"I\",\n\t\"uomLabel\":\"count\",\n\t\"defaultValue\":\"0\",\n\t\"formatting\":\"%s\",\n\t\"labels\":[\"count\",\"humans\"]\n}"
headers = {
    'cache-control': "no-cache",
    'Postman-Token': "d0a0021a-d1d4-4e60-bac7-1ed0c4fb1940"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

5c2ff80e53201a67a598bbdc


In [44]:
# Create the ValueDescriptor for the "dogcounter"
import requests

url = "http://localhost:48080/api/v1/valuedescriptor"

payload = "{\n    \"defaultValue\": \"0\",\n    \"description\": \"dog count\",\n    \"formatting\": \"%s\",\n    \"labels\": [\n        \"count\",\n        \"canines\"\n    ],\n    \"max\": \"100\",\n    \"min\": \"0\",\n    \"name\": \"caninecount\",\n    \"type\": \"I\",\n    \"uomLabel\": \"count\"\n}"
headers = {
    'cache-control': "no-cache",
    'Postman-Token': "30b43a6e-14b1-46fb-9cd7-2fc6ce7e0f4d"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

value descriptor 'caninecount' still referenced by readings



In [46]:
# Create the ValueDescriptor for the "depth"
import requests

url = "http://localhost:48080/api/v1/valuedescriptor"

payload = "{\n    \"defaultValue\": \"1\",\n    \"description\": \"scan distance\",\n    \"formatting\": \"%s\",\n    \"labels\": [\n        \"scan\",\n        \"distance\"\n    ],\n    \"max\": \"10\",\n    \"min\": \"1\",\n    \"name\": \"depth\",\n    \"type\": \"I\",\n    \"uomLabel\": \"feet\"\n}"
headers = {
    'cache-control': "no-cache",
    'Postman-Token': "04c48848-b798-40a4-8ab8-66559575955b"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

value descriptor 'depth' still referenced by readings



In [47]:
# Create the ValueDescriptor for the "duration"
import requests

url = "http://localhost:48080/api/v1/valuedescriptor"

payload = "{\n\t\"name\":\"duration\",\n\t\"description\":\"time between events\", \n\t\"min\":\"10\",\n\t\"max\":\"180\",\n\t\"type\":\"I\",\n\t\"uomLabel\":\"seconds\",\n\t\"defaultValue\":\"10\",\n\t\"formatting\":\"%s\",\n\t\"labels\":[\"duration\",\"time\"]\n}\n"
headers = {
    'cache-control': "no-cache",
    'Postman-Token': "26dc8757-f88e-45f4-a45e-eead6ad88478"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

5c2ff92753201a67a598c36c


In [48]:
# Create the ValueDescriptor for the "error"
import requests

url = "http://localhost:48080/api/v1/valuedescriptor"

payload = "{\n    \"defaultValue\": \"error\",\n    \"description\": \"error response message from a camera\",\n    \"formatting\": \"%s\",\n    \"labels\": [\n        \"error\",\n        \"message\"\n    ],\n    \"max\": \"\",\n    \"min\": \"\",\n    \"name\": \"cameraerror\",\n    \"type\": \"S\",\n    \"uomLabel\": \"\"\n}"
headers = {
    'cache-control': "no-cache",
    'Postman-Token': "c0c86617-67d4-4d98-bd44-657db122f10e"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

5c2ff96053201a67a598c4fb


## Define the Device

### Writing Data to the Core Data Service